In [1]:
from utils import read_from_jsonl
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

In [24]:
#train_data[0]
print(len(train_data))
print(len(test_data))
#test_data = test_data[:3000]

14041
3453


In [8]:
from copy import deepcopy
train_data_encoded = deepcopy(train_data)
test_data_encoded = deepcopy(test_data)

In [9]:
import numpy as np

curr_one_hot_encoding_mapping = {
    1: [1, 0, 0, 0],
    2: [0, 1, 0, 0],
    3: [0, 0, 1, 0],
    4: [0, 0, 0, 1]
}

prev_one_hot_encoding_mapping = {
    # 0 for start of string sequence
    0: [1, 0, 0, 0, 0],
    1: [0, 1, 0, 0, 0],
    2: [0, 0, 1, 0, 0],
    3: [0, 0, 0, 1, 0],
    4: [0, 0, 0, 0, 1]
}

for i, data in enumerate(train_data):
    for j, pos in enumerate(data["pos_tags"]):
        if j == 0:
            prev = prev_one_hot_encoding_mapping[0]
            curr = curr_one_hot_encoding_mapping[pos]
        else:
            prev = prev_one_hot_encoding_mapping[data["pos_tags"][j-1]]
            curr = curr_one_hot_encoding_mapping[pos]
        
        train_data_encoded[i]["pos_tags"][j] = np.array(prev + curr)
        

In [10]:
train_data_encoded[0]


{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': [1, 1, 1, 0, 1, 1, 1, 0, 1],
 'pos_tags': [array([1, 0, 0, 0, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1])]}

In [11]:
for i in train_data_encoded:
    chunk_tags = np.array(i["chunk_tags"])
    i["chunk_tags"] = chunk_tags
    pos_tags = np.array(i["pos_tags"])
    pos_tags.reshape(1, pos_tags.shape[0], pos_tags.shape[1])
    i["pos_tags"] = pos_tags


In [12]:
train_data_encoded[0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': array([1, 1, 1, 0, 1, 1, 1, 0, 1]),
 'pos_tags': array([[1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1]])}

In [13]:
for i in range(2):
    print(train_data_encoded[i]["pos_tags"].shape)
    print(train_data_encoded[i]["chunk_tags"].shape)

(9, 9)
(9,)
(2, 9)
(2,)


## Dataloader implementing Above things is in utils.py

In [77]:
import torch
from utils import read_from_jsonl
from utils import DataLoader
import matplotlib.pyplot as plt
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

train_data_loader = DataLoader(train_data[:3000], batch_size=len(train_data[:3000]))
test_data_loader = DataLoader(test_data, batch_size=1)

In [78]:
len(train_data), len(train_data_loader) 

(14041, 3000)

In [79]:
from utils import RecurrentPerceptron

a = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a.train_from_dataloader(train_data_loader, lr=0.05, nepochs=10, debug=False, print_interval=2)
#a.load_model('model.pkl')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.4419589042663574
Epoch 2 : Avg Loss : 0.43988585472106934
Epoch 4 : Avg Loss : 0.43972185254096985
Epoch 6 : Avg Loss : 0.4396650493144989
Epoch 8 : Avg Loss : 0.43963533639907837
Epoch 10 : Avg Loss : 0.43962517380714417


In [80]:
a.check_conditions()

(tensor(16), 16)

In [81]:
a.test_from_dataloader(test_data_loader, verbose=True)

  0%|          | 0/3453 [00:00<?, ?it/s]

tensor(0.8434)

In [82]:

# running this will overwrite the model file
a.save_model(path='/Users/keshav/Study/CS772/natural-language-processing-main 2/assignment-2', name='model_3000_10.pkl')

File already exists, overwriting


In [83]:
# final weights
a.W, a.V

(tensor([ 6.6279, -1.8724, -3.7012, -2.8595,  1.8169,  0.4002,  4.3571,  0.7107,
          5.5893]),
 tensor(-0.9459))

In [73]:
def k_fold_cross_validation(model, data, k=5, nepochs=10, batch_size=1):


    np.random.shuffle(data)

    fold_size = len(data) // k
    folds = [data[i:i+fold_size] for i in range(0, len(data), fold_size)]

    accuracies = []

    for i in range(k):
        validation_data = folds[i]
        train_data = [d for j, fold in enumerate(folds) if j != i for d in fold]
        print(len(train_data))
        #print(len())

        train_loader = DataLoader(train_data, batch_size=len(train_data))
        validation_loader = DataLoader(validation_data, batch_size=batch_size)

        model[i].train_from_dataloader(train_loader, lr=0.05, nepochs=10, debug=False, print_interval=2)
        print(model[i].check_conditions())

        accuracy = model[i].test_from_dataloader(validation_loader, verbose=False)
        accuracies.append(accuracy)

        print(f"Fold {i+1}/{k} - Validation Accuracy: {accuracy:.4f}")

    avg_accuracy = np.mean(accuracies)

    print(f"\nAverage Validation Accuracy: {avg_accuracy:.4f}")

    return avg_accuracy


In [74]:
import numpy as np
train_data = read_from_jsonl("train.jsonl")
train_data = train_data[:3000]

# Initialize the model
a1 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a2 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a3 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a4 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a5 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
model = [a1,a2,a3,a4,a5]
#a.train_from_dataloader(train_data_loader, lr=0.05, nepochs=10, debug=False, print_interval=2)

# Perform 5-fold cross-validation
avg_accuracy = k_fold_cross_validation(model, train_data, k=5, nepochs=10, batch_size=1)

print(f"\nOverall 5-Fold Cross-Validation Accuracy: {avg_accuracy:.4f}")

2400


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.4927491247653961
Epoch 2 : Avg Loss : 0.48600393533706665
Epoch 4 : Avg Loss : 0.48578062653541565
Epoch 6 : Avg Loss : 0.48571181297302246
Epoch 8 : Avg Loss : 0.48567891120910645
Epoch 10 : Avg Loss : 0.4856698215007782
Fold 1/5 - Validation Accuracy: 0.8958
2400


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.48415330052375793
Epoch 2 : Avg Loss : 0.4814966320991516
Epoch 4 : Avg Loss : 0.4812864065170288
Epoch 6 : Avg Loss : 0.48121923208236694
Epoch 8 : Avg Loss : 0.4811854958534241
Epoch 10 : Avg Loss : 0.4811752438545227
Fold 2/5 - Validation Accuracy: 0.8880
2400


...

Epoch 0 : Avg Loss : 0.48713016510009766
Epoch 2 : Avg Loss : 0.4767414331436157
Epoch 4 : Avg Loss : 0.4765324890613556
Epoch 6 : Avg Loss : 0.47646793723106384
Epoch 8 : Avg Loss : 0.4764396548271179
Epoch 10 : Avg Loss : 0.4764311611652374
Fold 3/5 - Validation Accuracy: 0.8841
2400


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.4918452501296997
Epoch 2 : Avg Loss : 0.48288294672966003
Epoch 4 : Avg Loss : 0.4826434254646301
Epoch 6 : Avg Loss : 0.4825671911239624
Epoch 8 : Avg Loss : 0.4825308620929718
Epoch 10 : Avg Loss : 0.4825195372104645
Fold 4/5 - Validation Accuracy: 0.9026
2400


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.4969436526298523
Epoch 2 : Avg Loss : 0.4907444715499878
Epoch 4 : Avg Loss : 0.49053719639778137
Epoch 6 : Avg Loss : 0.49047547578811646
Epoch 8 : Avg Loss : 0.49044886231422424
Epoch 10 : Avg Loss : 0.49044138193130493
Fold 5/5 - Validation Accuracy: 0.8999

Average Validation Accuracy: 0.8941


In [75]:
print(f"\nOverall 5-Fold Cross-Validation Accuracy: {avg_accuracy:.4f}")


Overall 5-Fold Cross-Validation Accuracy: 0.8941


In [76]:
import numpy as np
train_data = read_from_jsonl("train.jsonl")
train_data = train_data[:3000]

# Initialize the model
a1 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a2 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a3 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a4 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a5 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
model = [a1,a2,a3,a4,a5]
#a.train_from_dataloader(train_data_loader, lr=0.05, nepochs=10, debug=False, print_interval=2)

# Perform 5-fold cross-validation
avg_accuracy = k_fold_cross_validation(model, train_data, k=5, nepochs=10, batch_size=1)

print(f"\nOverall 5-Fold Cross-Validation Accuracy: {avg_accuracy:.4f}")

4800


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.6359727382659912
Epoch 2 : Avg Loss : 0.634477972984314
Epoch 4 : Avg Loss : 0.6344597339630127
Epoch 6 : Avg Loss : 0.6344596147537231
Epoch 8 : Avg Loss : 0.6344596147537231
Epoch 10 : Avg Loss : 0.634459912776947
Fold 1/5 - Validation Accuracy: 0.8551
4800


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.6487916111946106
Epoch 2 : Avg Loss : 0.6480486392974854
Epoch 4 : Avg Loss : 0.6480358839035034
Epoch 6 : Avg Loss : 0.6480362415313721
Epoch 8 : Avg Loss : 0.6480361223220825
Epoch 10 : Avg Loss : 0.6480361819267273
Fold 2/5 - Validation Accuracy: 0.8637
4800


...

Epoch 0 : Avg Loss : 0.6328361630439758
Epoch 2 : Avg Loss : 0.6344026327133179
Epoch 4 : Avg Loss : 0.6343903541564941
Epoch 6 : Avg Loss : 0.6343905329704285
Epoch 8 : Avg Loss : 0.634390652179718
Epoch 10 : Avg Loss : 0.6343905925750732
Fold 3/5 - Validation Accuracy: 0.8615
4800


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.6275531053543091
Epoch 2 : Avg Loss : 0.629525363445282
Epoch 4 : Avg Loss : 0.6295142769813538
Epoch 6 : Avg Loss : 0.6295143365859985
Epoch 8 : Avg Loss : 0.6295143365859985
Epoch 10 : Avg Loss : 0.629514217376709
Fold 4/5 - Validation Accuracy: 0.8545
4800


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.6363639831542969
Epoch 2 : Avg Loss : 0.6381193995475769
Epoch 4 : Avg Loss : 0.6381091475486755
Epoch 6 : Avg Loss : 0.6381092071533203
Epoch 8 : Avg Loss : 0.6381092071533203
Epoch 10 : Avg Loss : 0.6381093263626099
Fold 5/5 - Validation Accuracy: 0.8482

Average Validation Accuracy: 0.8566

Overall 5-Fold Cross-Validation Accuracy: 0.8566
